# An Introduction to Dynamic Mode Decomposition (DMD)

## Goal:

Approximate the leading eigencomposition of (high dimensional) linear operator $A \in \mathbb{C}^{n \times n}$ where $X'=AX$ to find spacial temporal coherant modes of the (possibly non-linear) system. 

In [28]:
import math #since torch doesn't have a built in pi function
import torch as pt
import numpy as np

pt.__version__

'1.7.1'

## Inputs to DMD

$n$ is the number of spatial points saved per shot and $m$ is the number of snapshots taken.

- Snapshot of fluid flow (long and skinny) $X = 
 \begin{bmatrix}
  \vert & \vert &     & \vert \\
  x_1   & x_2   & ... & x_{m-1} \\
  \vert & \vert &     & \vert
 \end{bmatrix} \in \mathbb{C}^{n \times (m-1)}$

- Snapshot of fluid flow evolved by one unit in time $X' = 
 \begin{bmatrix}
  \vert & \vert &     & \vert \\
  x_2   & x_3   & ... & x_{m} \\
  \vert & \vert &     & \vert
 \end{bmatrix} \in \mathbb{C}^{n \times (m-1)}$

 ### Inputs
 In this introduction, we will add together the following functions f1, f2, and f3 to create artificial test data:

In [14]:
#dmd wird kein img Anteil haben

#maybe do dmd in im and re, or change functions to re only

#img Funktionen austauschen mit reel (plot ~ gleich)

def f1(Xm, Tm, z):
    return pt.multiply(20-0.2*pt.pow(Xm, 2), pt.exp(z*Tm)).T

def f2(Xm, Tm, z):
    return pt.multiply(Xm, pt.exp(z*Tm)).T

def f3(Xm, Tm, z):
    return pt.multiply(10*pt.tanh(Xm/2)/pt.cosh(Xm/2), pt.exp(z*Tm)).T

### Meshgrid
A small meshgrid is created for calculating points of the functions. 

*Note: The inputs of Pytorch.meshgrid(t,x) are flipped compared to numpy.meshgrid(x,t). See [documentation](https://pytorch.org/docs/master/generated/torch.meshgrid.html).*

In [17]:
def create_mesh(x_start, x_end, n_x, t_start, t_end, n_t):
    x = pt.linspace(x_start, x_end, n_x)
    t = pt.linspace(t_start, t_end, n_t)
    return pt.meshgrid(t, x)

### Creating Data and calling DMD function

The mesh is created, the functions are combined, and the DMD function is called with a rank of 3.

Note: PyTorch has no built in Pi-Function, so we "create" our own. See [forum post](https://discuss.pytorch.org/t/np-pi-equivalent-in-pytorch/67157).

In [44]:
pt.pi = pt.acos(pt.zeros(1)).item()*2 # PyTorch has no built in Pi function

def main():
    Tm, Xm = create_mesh(-10, 10, 100, 0, 6*pt.pi, 80)
    X_1r = f1(Xm, Tm, -0.05+2.3j)
    X_2 = f2(Xm, Tm, 0.6j)
    X_3r = f3(Xm, Tm, 0.1+2.8j)
    data = X_1r + X_2 + X_3r

    rank=3
    dmd(data, rank=rank)

if __name__ == "__main__":
    main()

torch.complex64
torch.float32
torch.complex64


# DMD at a Glance:

**1. Compute Singular Value Decomposition (SVD) of big data matrix $X$ to find the dominant coherent structures (Columns of $U$).**

The exact SVD is reduced when, for example, 99% of the system energy is captured by the first $r$ comumns of $U$. The * denotes the conjugate transpose.

$$ X = U\Sigma V^* $$

**2. Project $A$ on the dominant singular vectors $U^*$ and $U$ to get the reduced dynamic operator $\tilde{A}$**

$X$ is replaced in $X' = AX$ with matrices from the SVD, which results in $ X' = AU\Sigma V^* $. Instead of doing a (very demanding) pseudo-inverse to find $A$, we project $\tilde{A}$ onto our dominant singluar vectors $U^*$ and $U$. $\tilde{A}$, which is only of the magnitude of time (much smaller than $A$), is a linear best fit dynamical system, that tell you how your POD modes evolve over time.

$$U^*X'V\Sigma^{-1} = U^*AU = \tilde{A}$$

**3. Compute the eigenvalues $\Lambda$ and eigenvectors $W$ of $\tilde{A}$**

$\tilde{A}$ has the same non-zero *eigenvalues* as $A$.

$$\tilde{A}W = W\Lambda$$

**4. Compute the eigenvectors $\Phi$ of $A$**

The eigenvectors $\Phi$ are also called modes.

$$\Phi = X'V\Sigma^{-1}W$$

### Singular Value Decomposition (SVD)

A function is defined to compute the SVD, which will be used for the DMD later. pt.svd() returns the reduced SVD by default.

pt.svd returns $V$, not $V*$ (conj. transposed), which is useful, since we need $V$ for step 4. $\Sigma$ will always be a real-valued, which is important for this example.

We truncate at 3 modes (Why?)

Columns of U are our POD modes. For more on POD, check out the [notebook](POD_introduction.ipynb).

*Note: torch.svd() is depricated. In torch 1.8, [torch.linalg.svd](https://pytorch.org/docs/1.8.0/linalg.html#torch.linalg.svd) will function like numpy.linalg.svd(), but this is not available in Version 1.7.1*

In [42]:
def svd(matrix, rank=None):
    U, s, V = pt.svd(matrix, some=False)  #returns V, not V*
    print(U.dtype)
    print(s.dtype)
    print(V.dtype)
    if rank is None:
        rank = s.shape[0]
    return U[:, :rank], s[:rank], V[:, :rank]

### Dynamic Mode Decomposition (DMD)

After finding the SVD of our matrix, we can find $\tilde{A} = U^*X'V\Sigma^{-1}$.

$\Sigma$ is a diagonal matrix and can therefore be inversed using 1/diag_elements.

After finding $\tilde{A}$, the eigenvalues and eigenvectors can be computed using pt.eig(). Since Pytorch does not currently support complex tensor operations (see [forum](https://discuss.pytorch.org/t/how-to-use-complex-eigenvalues/71983)), we will use np.linalg.eig() for this example.

Finally, the eigenvectors of A $\Phi$ are computed and returned together with the eigenvalues (same for A and At). 

In [43]:
def dmd(matrix, rank=None):
    U, s, V = svd(matrix[:,:-1], rank)
    s_inv = pt.diag(1.0/s).type(pt.complex64) #type cast since Sigma is real (not necessary when using only real data)
    
    At = U.conj().T @ matrix[:, 1:] @ V @ s_inv #At = torch.mm(torch.mm(torch.mm(U.conj().T, matrix[:, 1:]), V), s_inv) not necessary because @ works too
    
    val, vec = np.linalg.eig(At)
    #val, vec = torch.eig(At, eigenvectors=True)
    
    phi = matrix[:, 1:] @ V @ s_inv @ vec
    return (val, phi)

## Now what?


With our (spacial) modes $\Phi$ and (temporal) eigenvalues $\Lambda$, we can predict what the system will do in the future.

$$\hat{X}(k\Delta t) = \Phi\Lambda^kb_0$$

$\hat{X}$ is a future state prediction of $X$.

$\Lambda^k$ advances one time increment $\Delta t$ with each $k$.

$b_0$ is amplitude of modes. Condition for how much each mode is expressed in the data.

## Questions:

- use docstring with ''' notation, look at flowtorch/data/foamdataloader source code

- is rank of DMD always known? why do we truncate if we can do econ SVD? Can we find rank using dominant coherant structures of U? Is that what s.shape[0] does?

- is the "now what" correct?

## References

1. Kutz, J. N., Brunton, S. L. 1., Brunton, B. W., & Proctor, J. L. (2016). *Dynamic Mode Decomposition.* Philadelphia, PA, USA: Society for Industrial and Applied Mathematics.
2. Taylor, R. (2016) Dynamic Mode Decomposition in Python. *Pyrunner.* Accessed: 25 January 2021. http://www.pyrunner.com/weblog/2016/07/25/dmd-python/
    